In [13]:
import numpy as np

from color_model.base_color_model import BaseColorModel
from util.colorspace import XYZ2RGB, LMS2XYZ, DKL2LMS
DKL2RGB = XYZ2RGB @ LMS2XYZ @ DKL2LMS
RGB2DKL = np.linalg.inv(DKL2RGB)

ImportError: cannot import name 'DKL2LMS' from 'util.colorspace' (/home/budmonde/dev/nyu/vr-power-saver/util/colorspace.py)

In [14]:
model = BaseColorModel()
#0.73535698
rgb = np.array([0.5, 0.5, 0.5])
ecc = np.array([10])
resolution = 60

wh_idkl = model.compute_ellipses(rgb[None, :], ecc[None, :])

def ellipse_cov(w, h):
    # We fake an ellipse as an ellipsoid with epsilon depth
    return np.array([
        [1 / w**2,        0,             0],
        [0,        1 / h**2,             0],
        [0,               0, 1 / (1e-5)**2],
    ])

def get_ellipsoid_mesh(cov, centre, nu, nv):
    u, v = np.linspace(0, 1, nu), np.linspace(0, np.pi * 2, nv)
    x = np.outer(u, np.cos(v))
    y = np.outer(u, np.sin(v))
    z = np.outer(np.zeros_like(u), np.zeros_like(v))
    xyz = np.stack([x, y, z], 0).reshape(3, -1)
    
    eigval, eigvec = np.linalg.eigh(cov)
    # Ensure orientation of last eigvec is on same semi-sphere for all eigvecs
    if eigvec[:, 2] @ np.array([1, 0, 0]) < 0:
        eigvec[:, 2] = -eigvec[:, 2]
    # Ensure right-handed ellipse principle axes
    if np.cross(eigvec[:, 0], eigvec[:, 1]) @ eigvec[:, 2] > 0:
        a = 1 / np.sqrt(eigval[0]) * eigvec[:, 0]
        b = 1 / np.sqrt(eigval[1]) * eigvec[:, 1]
    else:
        a = 1 / np.sqrt(eigval[1]) * eigvec[:, 1]
        b = 1 / np.sqrt(eigval[0]) * eigvec[:, 0]
    c = 1 / np.sqrt(eigval[2]) * eigvec[:, 2]
    basis = np.stack([a, b, c])
    ellipse = basis @ xyz + centre.T
    return ellipse.reshape(3, *x.shape)
        
cov_idkl = ellipse_covariance(wh_idkl[0, 0], wh_idkl[0, 1])
cov_rgb = RGB2DKL.T @ cov_idkl @ RGB2DKL
mesh = get_ellipsoid_mesh(cov_rgb, rgb, resolution, resolution)

[0. 0. 0.]


NameError: name 'RGB2DKL' is not defined